# High Resolution inference 

In [ ]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ../..

In [ ]:
from videoseal.utils.display import save_img
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy
from videoseal.augmentation import Identity, JPEG
from videoseal.modules.jnd import JND

import os
import omegaconf
from tqdm import tqdm
import gc
from PIL import Image

import torch
import torchvision

to_tensor = torchvision.transforms.ToTensor()
to_pil = torchvision.transforms.ToPILImage()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu" 

In [9]:
# Directory containing videos
assets_dir = "/checkpoint/pfz/projects/videoseal/assets/imgs"
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

# Checkpoint
ckpts = {
    # "hidden": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.0_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=hidden/checkpoint.pth',
    # "unet": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.5_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=unet_small2/checkpoint.pth',
    # "scaling_laws_smalldetector_tinyembedder":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1105-videoseal0.2-scalinglaws/_lambda_d=0.0_extractor_model=sam_small_embedder_model=0/checkpoint.pth",
    # "scaling_laws_tinydetector_tinyembedder":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1105-videoseal0.2-scalinglaws/_lambda_d=0.0_extractor_model=sam_tiny_embedder_model=0/checkpoint.pth",
    # "JND_fix_discloss":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1108-videoseal0.2-discloss-fix-removeunused-params/_attenuation=jnd_3_3_nbits=64_lambda_d=0.5_video_start=100/checkpoint.pth",
    # "1111_discloss_sleepwake_highssim":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1109-videoseal0.2-discloss-fix-hing-sleepwake-4nodes/_scaling_w=0.1_lambda_i=0.25_disc_hinge_on_logits_fake=False_sleepwake=True_video_start=500/checkpoint.pth"
    # "1111-finetuned":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1111-videoseal0.2-sleepwake-resume/_attenuation=jnd_1_1/checkpoint.pth"
    # "1112-videoseal0.2":"/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1111-videoseal0.2-archsearch-4nodes/_attenuation=None_nbits=64_finetune_detector_start=1000_embedder_model=unet_small2_quant/checkpoint.pth",
    # "1118-yuv-400":"/checkpoint/pfz/2024_logs/1118_vseal_long_sab/_scheduler=0_optimizer=adopt,lr=1e-5/checkpoint400.pth",
    # "1118-yuv-800":"/checkpoint/pfz/2024_logs/1118_vseal_long_sab/_scheduler=1_optimizer=AdamW,lr=1e-5/checkpoint.pth",
    # "trustmark": "baseline/trustmark",
    # "videoseal0.2b": "/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1111-videoseal0.2-archsearch-4nodes/_attenuation=None_nbits=64_finetune_detector_start=800_embedder_model=unet_small2_quant/checkpoint.pth",
    # "videoseal0.2a": "/private/home/hadyelsahar/work/code/videoseal/2024_logs_large-exp/1109-videoseal0.2-discloss-fix-hing-sleepwake-4nodes/_scaling_w=0.5_lambda_i=0.5_disc_hinge_on_logits_fake=True_sleepwake=False_video_start=500/checkpoint.pth",
    # "videoseal0.4": "/large_experiments/meres/hadyelsahar/2024_logs/1120-videoseal0.4/_scaling_w=0.5_sleepwake=False_videowam_step_size=4_extractor_model=sam_tiny/checkpoint.pth",
    # "wam": "baseline/wam",
    "cin": "baseline/cin",
    # "mbrs": "baseline/mbrs",
}

for ckpt_name, ckpt_path in ckpts.items():
    # ckpt_path = "/checkpoint/pfz/2024_logs/1028_vseal_long/_seed=3_optimizer=AdamW,lr=1e-4_embedder_model=unet_small2_yuv/checkpoint600.pth"

    # a timer to measure the time
    timer = Timer()

    # Iterate over all checkpoints
    wam = setup_model_from_checkpoint(ckpt_path)
    wam.eval()
    wam.to(device)

    # Iterate over all video files in the directory
    files = [f for f in os.listdir(assets_dir) if f.endswith(".png")]
    files = [os.path.join(assets_dir, f) for f in files]

    for file in tqdm(files, desc=f"Processing Images"):
        # load image
        imgs = Image.open(file, "r").convert("RGB")  # keep only rgb channels
        imgs = to_tensor(imgs).unsqueeze(0).float()

        # Watermark embedding
        timer.start()
        outputs = wam.embed(imgs, is_video=False)
        torch.cuda.synchronize()
        print(f"embedding watermark  - took {timer.stop():.2f}s")

        # compute diff
        imgs_w = outputs["imgs_w"]  # b c h w
        msgs = outputs["msgs"]  # b k
        diff = imgs_w - imgs

        # save
        timer.start()
        base_save_name = os.path.join(output_dir, os.path.basename(file).replace(".png", ""))
        print(f"saving videos to {base_save_name}")
        save_img(imgs[0], f"{base_save_name}_ori.png")
        save_img(imgs_w[0], f"{base_save_name}_wm.png")
        save_img(10*diff[0].abs(), f"{base_save_name}_diff.png")

        # Compute min and max values, reshape, and normalize
        min_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).min(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        max_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).max(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        normalized_images = (diff - min_vals) / (max_vals - min_vals)

        # Save the normalized video
        save_img(normalized_images[0], f"{base_save_name}_diff_norm.png")
        print(f"saving videos - took {timer.stop():.2f}s")

        # Augment video
        print(f"compressing and detecting watermarks")
        for qf in [80, 40]:
            imgs_aug, _ = JPEG()(imgs_w, None,qf)

            # detect
            timer.start()
            outputs = wam.detect(imgs_aug, is_video=True)
            preds = outputs["preds"]
            print(preds)
            bit_preds = preds[:, 1:]  # b k ...
            bit_accuracy_ = bit_accuracy(
                bit_preds,
                msgs
            ).nanmean().item()
            print(f"bit accuracy at JPEG {qf} is {bit_accuracy_:.2f} - took {timer.stop():.2f}s")

        del outputs, imgs, imgs_w, diff, min_vals, max_vals, normalized_images

    # Free model from GPU
    del wam
    torch.cuda.empty_cache()

Processing Images:   0%|          | 0/4 [00:00<?, ?it/s]

embedding watermark  - took 0.38s
saving videos to outputs/00
saving videos - took 1.51s
compressing and detecting watermarks
tensor([[ 0.0000,  0.9498,  0.9816,  0.9610,  0.9585, -0.9406, -0.9424, -0.9920,
          0.8935,  0.9385,  0.9679, -0.9509, -0.9353, -0.9915, -0.9152, -0.9057,
         -0.9686, -0.9503,  1.0000,  0.9371, -0.9873, -0.9805, -0.9527, -0.9509,
         -0.9942,  0.9484, -0.9855, -0.9672,  0.9832,  0.9626,  0.9751]])
bit accuracy at JPEG 80 is 1.00 - took 0.67s


Processing Images:   0%|          | 0/4 [00:09<?, ?it/s]


KeyboardInterrupt: 

## With attenuation

In [ ]:
# Directory containing videos
assets_dir = "assets/imgs"
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Checkpoint
ckpt_path = "/checkpoint/pfz/2024_logs/1028_vseal_long/_seed=3_optimizer=AdamW,lr=1e-4_embedder_model=unet_small2_yuv/checkpoint600.pth"

# a timer to measure the time
timer = Timer()

# Iterate over all checkpoints
wam = setup_model_from_checkpoint(ckpt_path)
wam.eval()
wam.to(device)

# create attenuation
attenuation_cfg = "configs/attenuation.yaml"
attenuation = "jnd_1_1"
attenuation_cfg = omegaconf.OmegaConf.load(attenuation_cfg)[attenuation]
attenuation = JND(**attenuation_cfg)
wam.attenuation = attenuation
wam.scaling_w = 0.2

# Iterate over all video files in the directory
files = [f for f in os.listdir(assets_dir) if f.endswith(".png")]
files = [os.path.join(assets_dir, f) for f in files]

for file in tqdm(files, desc=f"Processing Videos"):
    # load image
    imgs = Image.open(file, "r").convert("RGB")  # keep only rgb channels
    imgs = to_tensor(imgs).unsqueeze(0).float()

    # Watermark embedding
    timer.start()
    outputs = wam.embed(imgs, is_video=False)
    # torch.cuda.synchronize()
    # print(f"embedding watermark  - took {timer.stop():.2f}s")

    # compute diff
    imgs_w = outputs["imgs_w"]  # b c h w
    msgs = outputs["msgs"]  # b k
    diff = imgs_w - imgs

    # save
    timer.start()
    base_save_name = os.path.join(output_dir, os.path.basename(file).replace(".png", ""))
    save_img(imgs[0], f"{base_save_name}_ori.png")
    save_img(imgs_w[0], f"{base_save_name}_wm.png")
    save_img(diff[0], f"{base_save_name}_diff.png")

    # Compute min and max values, reshape, and normalize
    min_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).min(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
    max_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).max(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
    normalized_images = (diff - min_vals) / (max_vals - min_vals)

    # Save the normalized video
    save_img(normalized_images[0], f"{base_save_name}_diff_norm.png")

    # Augment video
    for qf in [80, 40]:
        imgs_aug, _ = JPEG()(imgs_w, None,qf)

        # detect
        timer.start()
        outputs = wam.detect(imgs_aug, is_video=True)
        preds = outputs["preds"]
        bit_preds = preds[:, 1:]  # b k ...
        bit_accuracy_ = bit_accuracy(
            bit_preds,
            msgs
        ).nanmean().item()
        print(f"bit accuracy at JPEG {qf} is {bit_accuracy_:.2f} - took {timer.stop():.2f}s")

    del outputs, imgs, imgs_w, diff, min_vals, max_vals, normalized_images

# Free model from GPU
del wam
torch.cuda.empty_cache()